### 영상에서 마스크 착용 여부 판별하기

In [1]:
import tensorflow as tf
tf.__version__

'2.6.0'

In [8]:
import cv2
from tensorflow.keras.models import load_model
import numpy as np

In [6]:
mask_model = load_model("./mask_detect_model2.h5")

In [10]:
# 얼굴검출 모델 사용하기
face_model = cv2.CascadeClassifier(cv2.data.haarcascades+"haarcascade_frontalface_default.xml")

#반복문 이용해서 영상 출력하기
try : 
    print("영상로딩을 시작합니다..")
    video_cap = cv2.VideoCapture("./03.mp4")
    #video_cap = cv2.VideoCapture(0) 0으로 작성시 컴퓨터와 연결된 카메라 연동
    
    while True:
        ret,frame = video_cap.read()
        
        if ret :
            # 얼굴검출 코드 추가
            faces = face_model.detectMultiScale(frame, 1.2,8)
            for(x,y,w,h) in faces:
                roi = frame[y:y+h,x:x+w]
                # BGR -> RGB
                roi_RGB = cv2.cvtColor(roi,cv2.COLOR_BGR2RGB)
                # resize
                roi_resize = cv2.resize(roi_RGB,dsize=(224,224),
                                       interpolation=cv2.INTER_AREA)
                # -1 ~ 1
                roi_normal = (np.array(roi_resize,dtype=np.float32)/127.0) - 1
                # predict
                pre = mask_model.predict(roi_normal.reshape(1,224,224,3))
                #print(pre)
                if pre[0] >= 0.5:
                    cv2.rectangle(frame,(x,y),(x+w,y+h),(0,0,255),3)
                else:
                    cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),3)
                
            cv2.imshow("video",frame)
            k = cv2.waitKey(10) # 초당 프레임수를 맞추기 위함 : 1000/24
        else:
            break;
            
        if k == 27 : #27번은 ESC
            break;
            
    cv2.destroyAllWindows()
    video_cap.release()
except :
    print("영상로딩 실패")

영상로딩을 시작합니다..
